In [1]:
#importing all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib.pyplot import *
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

In [1]:
#Loading a dataframe for working
#data should contain 48 temperature measurement events, i.e. 24 prior to triggered alarm and 24 afterwards
#with a sampling rate of 10 minutes in this project, it corresponds to a time frame of eight hours
al = pd.read_csv('example_data.csv', sep = ';')

In [2]:
#defining functions
#function for fitting and prediction
def temp(x, T0, k, Ta):
    return (T0 - Ta)*np.exp(-k * x) + Ta
#function for plotting
def temp_plot(xv, T0v, kv, Tav):
    return (T0v - Tav)*np.exp(-kv * xv) + Tav
#function for estimation of initial temperature (T0) with the help of Kalman filter
def kalm(err_act, err_orig, measurement_orig, measurements): #err_act - actual measurement error, err_orig - orininal error estimate, measurements - vector of measurements with stable tamperature within measurement error, measurement_orig - original measurement estimate
    updated_measurements = [] #list for storing updated T0 estimates
    errors = [err_orig] #list with error estimates
    for measurement in measurements:
        kalman_gain = errors[-1]/(errors[-1] + err_act)
        updated_measurements.append(measurement_orig + kalman_gain*(measurement - measurement_orig))
        err_cur = (1 - kalman_gain)*errors[-1]
        errors.append(err_cur)
    T0 = updated_measurements[-1]
    return T0

In [4]:
#Combining the code for fitting and plotting with the logic for temperature prediction text
#looping through all vectors with measurements in example_data
for m in range(al):
    test_array = al.iloc[m, :]
    input_y = np.array(test_array)
    for n in range(len(input_y)):
        input_y[n] = round(input_y[n], 2)
    input_yp = input_y #probably not necessary; got added in the course of multiple changes and deletions in the course of development

    y_temp = input_y[0:3] #taking initial readings
    y_temp = list(y_temp) #changing type
    x_val = np.arange(48) #x-scale vector for the entire horizon of prediction
    x_valp = x_val #again, probably not necessary; a rudiment as a result of multiple changes
    x_val_p = np.arange(len(y_temp)) #x-scale vector for fitting
    x_val_p = list(x_val_p) #changing type
    T0 = y_temp[0] #T0 initialization
    last_sign = [] #signs list
    last_slope = [] #slope list
    plot_dots = len(x_val_p) - 1 #number of dots to plot
    lower = lower #lower threshold (should be either in example_data if different across measurements or specified once for all case)
    higher = higher #higher threshold (same conditions apply as to lower threshold)

    #appending lists for slope and sign for further evaluations of conditions for fitting and plotting
    for i in range(2, len(input_y)-1):
        if abs(input_y[i] - input_y[i-1]) > abs(input_y[i-1] - input_y[i-2]):
            slope = 1
        else:
            slope = -1
        last_slope.append(slope)
        if input_y[i] > input_y[i-1] and input_y[i-1] > input_y[i-2]:
            sign_last = 1
            sign_current = 1
        elif input_y[i] < input_y[i-1] and input_y[i-1] < input_y[i-2]:
            sign_last = -1
            sign_current = -1
        elif input_y[i] > input_y[i-1] and input_y[i-1] < input_y[i-2]:
            sign_last = -1
            sign_current = 1
        elif input_y[i] < input_y[i-1] and input_y[i-1] > input_y[i-2]:
            sign_last = 1
            sign_current = -1
        elif input_y[i] == input_y[i-1] and input_y[i-1] > input_y[i-2]:
            sign_last = 1
            sign_current = 1
        elif input_y[i] == input_y[i-1] and input_y[i-1] < input_y[i-2]:
            sign_last = -1
            sign_current = -1
        elif input_y[i] > input_y[i-1] and input_y[i-1] == input_y[i-2]:
            sign_last = 1
            sign_current = 1
        elif input_y[i] < input_y[i-1] and input_y[i-1] == input_y[i-2]:
            sign_last = -1
            sign_current = -1
        elif input_y[i] == input_y[i-1] and input_y[i-1] == input_y[i-2]:
            sign_last = 0
            sign_current = 0
        else:
            print("I have missed something when assigning signs") #this condition only for error tracing
            
        if len(last_sign) >= 2:
            if last_sign[-1] == 0 and sign_current == 0:
                last_sign.append(sign_current)
            elif last_sign[-1] == 0 and sign_current != 0:
                last_sign[-1] = sign_current
                last_sign.append(sign_current)
            elif last_sign[-1] != 0 and sign_current == 0:
                last_sign.append(last_sign[-1])
            elif last_sign[-1] != 0 and sign_current != 0:
                last_sign.append(sign_current)
            else:
                print("Something went wrong for the last_sign length over 2") #this condition only for error tracing
        elif len(last_sign) < 2:
            last_sign.append(sign_last)
            last_sign.append(sign_current)
        else:
            print("I have missed something when appending the sign list") #this condition only for error tracing

        #now, using sloples and signs for evaluating thermal system stability, so that we can decide upon the number of
        #previous measurements and initial temperature variable for the formula
        if last_slope[-1] == 1 and last_sign[-2] != last_sign[-1]:
            y_temp = y_temp[-2:]
            T0 = y_temp[0]
            x_val_p = np.arange(len(y_temp))
            x_val_p = list(x_val_p)
            input_yp = input_yp[(plot_dots - 1):]
            x_valp = x_valp[:-(plot_dots - 1)]
            #first - fitting, then - plotting (with decision-support text)
            try:
                popt, pcov = curve_fit(lambda x, k, Ta: temp(x, T0, k, Ta), x_val_p, y_temp, bounds = ([0, -10], [2, 60]), max_nfev = 1000)
                plt.figure(figsize = (10, 7))
                plt.plot(x_valp[0:len(x_val_p)], input_yp[0:len(x_val_p)], 'yo', label = 'Temperature changed trajectory and diverges')
                plt.plot(x_valp[len(x_val_p):], input_yp[len(x_val_p):], 'bo')
                plt.xlabel('Measurement index')
                plt.ylabel('Temperature')
                plt.axhline(y = lower, color = 'red', linestyle = '--')
                plt.axhline(y = higher, color = 'red', linestyle = '--')
                plt.plot(x_val_p, temp_plot(np.array(x_val_p), T0, *popt), 'g-', label = 'Parameter estimation: k=%5.3f, Ta=%5.3f' % tuple(popt))
                x_val_pp = x_val_p[:]
                for j in range(1, 7):
                    x_val_pp.append(x_val_p[-1] + j)
                
                #different versions of decision-support text depending on the quality of prediction
                if len(x_val_p) == 2:
                    label_to_add = '(Unreliable prediction due to a small number of fitting points)\n'
                elif len(x_val_p) == 3:
                    label_to_add = '(Relatively reliable prediction)\n'
                elif len(x_val_p) > 3:
                    label_to_add = '(Reliable prediction)\n'
                else:
                    print('Number of points in x_val_p is unexpectedly below 2 (alarm ID ', al.iloc[m, 0], ')') #this condition only for error tracing
                
                #below: variables and conditions for further decision-support text regarding convergence
                y_pred = []
                dev_lower = 0
                dev_higher = 0
                dev_lower_list = []
                dev_higher_list = []
                range_val = 0
                range_list = []
                for o in range(len(x_val_p), len(x_val_pp)):
                    y_pred.append(popt[1] + (T0 - popt[1])*np.exp(-popt[0]*o))
                    if y_pred[-1] >= higher:
                        dev_higher = 10*(o - len(x_val_p) + 1)
                        dev_higher_list.append(dev_higher)
                    elif y_pred[-1] <= lower:
                        dev_lower = 10*(o - len(x_val_p) + 1)
                        dev_lower_list.append(dev_lower)
                    elif y_pred[-1] < higher and y_pred[-1] > lower:
                        range_val = 10*(o - len(x_val_p) + 1)
                        range_list.append(range_val)
                    else:
                        print('Something went wrong with the comparison of predicted temperature with the thresholds') #this condition only for error tracing
                if len(range_list) == 0:
                    range_list.append(0)
                if len(dev_lower_list) == 0:
                    dev_lower_list.append(0)
                if len(dev_higher_list) == 0:
                    dev_higher_list.append(0)
                if min(range_list) > max(dev_lower_list) and min(range_list) > max(dev_higher_list): #temperature will be within range
                    if (len(dev_lower_list) == 1 and dev_lower_list[0] == 0) and (len(dev_higher_list) == 1 and dev_higher_list[0] == 0):
                        label_to_add += 'Temperature is expected to remain within the range within the next 60 minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                    else:
                        label_to_add = 'Temperature is expected to be within range in ' + str(min(range_list)) + ' minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                elif min(dev_lower_list) > max(range_list) and min(dev_lower_list) > max(dev_higher_list):
                    if ((len(range_list) == 1 and range_list[0] == 0) and (len(dev_higher_list) == 1 and dev_higher_list[0] == 0)):
                        label_to_add += 'Temperature is expected to remain below lower threshold within the next 60 minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                    else:
                        label_to_add += 'Temperature is expected to go below lower threshold in ' + str(min(dev_lower_list)) + ' minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                elif min(dev_higher_list) > max(dev_lower_list) and min(dev_higher_list) > max(range_list):
                    if ((len(dev_lower_list) == 1 and dev_lower_list[0] == 0) and (len(range_list) == 1 and range_list[0] == 0)):
                        label_to_add += 'Temperature is expected to remain beyond higher threshold within the next 60 minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                    else:
                        label_to_add += 'Temperature is expected to go beyond higher threshold in ' + str(min(dev_higher_list)) + ' minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                
                plt.plot(x_val_pp[(len(x_val_p)):], temp_plot(np.array(x_val_pp), T0, *popt)[(len(x_val_p)):], 'r^', label = label_to_add)
                plt.legend(loc = 'best')
                plt.suptitle('Points processed in total: ' + str(len(input_y[:(i+1)])) + '; alarm ID: ' + str(al.iloc[m, 0]))
                plt.savefig('AlarmID %i Points %i.png' % (al.iloc[m, 0], len(input_y[:(i+1)])), dpi=100)
                plt.close()
                plt.rcParams.update({'figure.max_open_warning': 0})
            except RuntimeError:
                print('Optimal paramteres not found for ', y_temp)   
            #necessary updates to the input vectors
            y_temp.append(input_y[i+1])
            x_val_p = np.arange(len(y_temp))
            x_val_p = list(x_val_p)
            plot_dots = 2
            
            #in the chunks of code below, no comments will be duplicated; all comments for analogous sections above hold true
            
        elif last_slope[-1] == 1 and last_sign[-2] == last_sign[-1]:
            y_temp = y_temp[-2:]
            T0 = y_temp[0]
            x_val_p = np.arange(len(y_temp))
            x_val_p = list(x_val_p)
            input_yp = input_yp[(plot_dots - 1):]
            x_valp = x_valp[:-(plot_dots - 1)]
            try:
                popt, pcov = curve_fit(lambda x, k, Ta: temp(x, T0, k, Ta), x_val_p, y_temp, bounds = ([0, -10], [2, 60]), max_nfev = 1000)
                plt.figure(figsize = (10, 7))
                plt.plot(x_valp[0:len(x_val_p)], input_yp[0:len(x_val_p)], 'yo', label = 'Temperature diverges')
                plt.plot(x_valp[len(x_val_p):], input_yp[len(x_val_p):], 'bo')
                plt.xlabel('Measurement index')
                plt.ylabel('Temperature')
                plt.axhline(y = lower, color = 'red', linestyle = '--')
                plt.axhline(y = higher, color = 'red', linestyle = '--')
                plt.plot(x_val_p, temp_plot(np.array(x_val_p), T0, *popt), 'g-', label = 'Parameter estimation: k=%5.3f, Ta=%5.3f' % tuple(popt))
                x_val_pp = x_val_p[:]
                for j in range(1, 7):
                    x_val_pp.append(x_val_p[-1] + j)
                
                if len(x_val_p) == 2:
                    label_to_add = '(Unreliable prediction due to a small number of fitting points)\n'
                elif len(x_val_p) == 3:
                    label_to_add = '(Relatively reliable prediction)\n'
                elif len(x_val_p) > 3:
                    label_to_add = '(Reliable prediction)\n'
                else:
                    print('Number of points in x_val_p is unexpectedly below 2 (alarm ID ', al.iloc[m, 0], ')') #this condition only for error tracing
                y_pred = []
                dev_lower = 0
                dev_higher = 0
                dev_lower_list = []
                dev_higher_list = []
                range_val = 0
                range_list = []
                for o in range(len(x_val_p), len(x_val_pp)):
                    y_pred.append(popt[1] + (T0 - popt[1])*np.exp(-popt[0]*o))
                    if y_pred[-1] >= higher:
                        dev_higher = 10*(o - len(x_val_p) + 1)
                        dev_higher_list.append(dev_higher)
                    elif y_pred[-1] <= lower:
                        dev_lower = 10*(o - len(x_val_p) + 1)
                        dev_lower_list.append(dev_lower)
                    elif y_pred[-1] < higher and y_pred[-1] > lower:
                        range_val = 10*(o - len(x_val_p) + 1)
                        range_list.append(range_val)
                    else:
                        print('Something went wrong with the comparison of predicted temperature with the thresholds') #this condition only for error tracing
                if len(range_list) == 0:
                    range_list.append(0)
                if len(dev_lower_list) == 0:
                    dev_lower_list.append(0)
                if len(dev_higher_list) == 0:
                    dev_higher_list.append(0)
                if min(range_list) > max(dev_lower_list) and min(range_list) > max(dev_higher_list): #temperature will be within range
                    if (len(dev_lower_list) == 1 and dev_lower_list[0] == 0) and (len(dev_higher_list) == 1 and dev_higher_list[0] == 0):
                        label_to_add += 'Temperature is expected to remain within the range within the next 60 minutes'
                        if abs(y_pred[-1] - higher) <= 0.5:
                            label_to_add += ',\nhowever, it may be very close to higher threshold'
                        elif abs(y_pred[-1] - lower) <= 0.5:
                            label_to_add += ',\nhowever, it may be very close to lower threshold'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                    else:
                        label_to_add += 'Temperature is expected to be within range in ' + str(min(range_list)) + ' minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                elif min(dev_lower_list) > max(range_list) and min(dev_lower_list) > max(dev_higher_list):
                    if ((len(range_list) == 1 and range_list[0] == 0) and (len(dev_higher_list) == 1 and dev_higher_list[0] == 0)):
                        label_to_add += 'Temperature is expected to remain below lower threshold within the next 60 minutes'
                        if popt[-1] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                    else:
                        label_to_add += 'Temperature is expected to go below lower threshold in ' + str(min(dev_lower_list)) + ' minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                elif min(dev_higher_list) > max(dev_lower_list) and min(dev_higher_list) > max(range_list):
                    if ((len(dev_lower_list) == 1 and dev_lower_list[0] == 0) and (len(range_list) == 1 and range_list[0] == 0)):
                        label_to_add += 'Temperature is expected to remain beyond higher threshold within the next 60 minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                    else:
                        label_to_add += 'Temperature is expected to go beyond higher threshold in ' + str(min(dev_higher_list)) + ' minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                
                plt.plot(x_val_pp[(len(x_val_p)):], temp_plot(np.array(x_val_pp), T0, *popt)[(len(x_val_p)):], 'r^', label = label_to_add)
                plt.legend(loc = 'best')
                plt.suptitle('Points processed in total: ' + str(len(input_y[:(i+1)])) + '; alarm ID: ' + str(al.iloc[m, 0]))
                plt.savefig('AlarmID %i Points %i.png' % (al.iloc[m, 0], len(input_y[:(i+1)])), dpi=100)
                plt.close()
                plt.rcParams.update({'figure.max_open_warning': 0})
            except RuntimeError:
                print('Optimal paramteres not found for ', y_temp)
            y_temp.append(input_y[i+1])
            x_val_p = np.arange(len(y_temp))
            x_val_p = list(x_val_p)
            plot_dots = 2
        elif last_slope[-1] == -1 and last_sign[-2] == last_sign[-1]:
            T0 = y_temp[0]
            try:
                popt, pcov = curve_fit(lambda x, k, Ta: temp(x, T0, k, Ta), x_val_p, y_temp, bounds = ([0, -10], [2, 60]), max_nfev = 1000)
                plt.figure(figsize = (10, 7))
                plt.plot(x_valp[0:len(x_val_p)], input_yp[0:len(x_val_p)], 'yo', label = 'Temperature converges')
                plt.plot(x_valp[len(x_val_p):], input_yp[len(x_val_p):], 'bo')
                plt.xlabel('Measurement index')
                plt.ylabel('Temperature')
                plt.axhline(y = lower, color = 'red', linestyle = '--')
                plt.axhline(y = higher, color = 'red', linestyle = '--')
                plt.plot(x_val_p, temp_plot(np.array(x_val_p), T0, *popt), 'g-', label = 'Parameter estimation: k=%5.3f, Ta=%5.3f' % tuple(popt))
                x_val_pp = x_val_p[:]
                for j in range(1, 7):
                    x_val_pp.append(x_val_p[-1] + j)
                
                if len(x_val_p) == 2:
                    label_to_add = '(Unreliable prediction due to a small number of fitting points)\n'
                elif len(x_val_p) == 3:
                    label_to_add = '(Relatively reliable prediction)\n'
                elif len(x_val_p) > 3:
                    label_to_add = '(Reliable prediction)\n'
                else:
                    print('Number of points in x_val_p is unexpectedly below 2 (alarm ID ', al.iloc[m, 0], ')') #this condition only for error tracing
                y_pred = []
                dev_lower = 0
                dev_higher = 0
                dev_lower_list = []
                dev_higher_list = []
                range_val = 0
                range_list = []
                for o in range(len(x_val_p), len(x_val_pp)):
                    y_pred.append(popt[1] + (T0 - popt[1])*np.exp(-popt[0]*o))
                    if y_pred[-1] >= higher:
                        dev_higher = 10*(o - len(x_val_p) + 1)
                        dev_higher_list.append(dev_higher)
                    elif y_pred[-1] <= lower:
                        dev_lower = 10*(o - len(x_val_p) + 1)
                        dev_lower_list.append(dev_lower)
                    elif y_pred[-1] < higher and y_pred[-1] > lower:
                        range_val = 10*(o - len(x_val_p) + 1)
                        range_list.append(range_val)
                    else:
                        print('Something went wrong with the comparison of predicted temperature with the thresholds') #this condition only for error tracing
                if len(range_list) == 0:
                    range_list.append(0)
                if len(dev_lower_list) == 0:
                    dev_lower_list.append(0)
                if len(dev_higher_list) == 0:
                    dev_higher_list.append(0)
                if min(range_list) > max(dev_lower_list) and min(range_list) > max(dev_higher_list): #temperature will be within range
                    if (len(dev_lower_list) == 1 and dev_lower_list[0] == 0) and (len(dev_higher_list) == 1 and dev_higher_list[0] == 0):
                        label_to_add += 'Temperature is expected to remain within the range within the next 60 minutes'
                        if abs(y_pred[-1] - higher) <= 0.5:
                            label_to_add += ',\nhowever, it may be very close to higher threshold'
                        elif abs(y_pred[-1] - lower) <= 0.5:
                            label_to_add += ',\nhowever, it may be very close to lower threshold'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                    else:
                        label_to_add += 'Temperature is expected to be within range in ' + str(min(range_list)) + ' minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                elif min(dev_lower_list) > max(range_list) and min(dev_lower_list) > max(dev_higher_list):
                    if ((len(range_list) == 1 and range_list[0] == 0) and (len(dev_higher_list) == 1 and dev_higher_list[0] == 0)):
                        label_to_add += 'Temperature is expected to remain below lower threshold within the next 60 minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                    else:
                        label_to_add += 'Temperature is expected to go below lower threshold in ' + str(min(dev_lower_list)) + ' minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                elif min(dev_higher_list) > max(dev_lower_list) and min(dev_higher_list) > max(range_list):
                    if ((len(dev_lower_list) == 1 and dev_lower_list[0] == 0) and (len(range_list) == 1 and range_list[0] == 0)):
                        label_to_add += 'Temperature is expected to remain beyond higher threshold within the next 60 minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                    else:
                        label_to_add += 'Temperature is expected to go beyond higher threshold in ' + str(min(dev_higher_list)) + ' minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                
                plt.plot(x_val_pp[(len(x_val_p)):], temp_plot(np.array(x_val_pp), T0, *popt)[(len(x_val_p)):], 'r^', label = label_to_add)
                plt.legend(loc = 'best')
                plt.suptitle('Points processed in total: ' + str(len(input_y[:(i+1)])) + '; alarm ID: ' + str(al.iloc[m, 0]))
                plt.savefig('AlarmID %i Points %i.png' % (al.iloc[m, 0], len(input_y[:(i+1)])), dpi=100)
                plt.close()
                plt.rcParams.update({'figure.max_open_warning': 0})
            except RuntimeError:
                print('Optimal paramteres not found for ', y_temp)
            y_temp.append(input_y[i+1])
            x_val_p = np.arange(len(y_temp))
            x_val_p = list(x_val_p)
            plot_dots += 1
        elif last_slope[-1] == -1 and last_sign[-2] != last_sign[-1]:
            y_temp = y_temp[-2:]
            T0 = y_temp[0]
            x_val_p = np.arange(len(y_temp))
            x_val_p = list(x_val_p)
            input_yp = input_yp[(plot_dots - 1):]
            x_valp = x_valp[:-(plot_dots - 1)]
            try:
                popt, pcov = curve_fit(lambda x, k, Ta: temp(x, T0, k, Ta), x_val_p, y_temp, bounds = ([0, -10], [2, 60]), max_nfev = 1000)
                plt.figure(figsize = (10, 7))
                plt.plot(x_valp[0:len(x_val_p)], input_yp[0:len(x_val_p)], 'yo', label = 'Temperature changed trajectory')
                plt.plot(x_valp[len(x_val_p):], input_yp[len(x_val_p):], 'bo')
                plt.xlabel('Measurement index')
                plt.ylabel('Temperature')
                plt.axhline(y = lower, color = 'red', linestyle = '--')
                plt.axhline(y = higher, color = 'red', linestyle = '--')
                plt.plot(x_val_p, temp_plot(np.array(x_val_p), T0, *popt), 'g-', label = 'Parameter estimation: k=%5.3f, Ta=%5.3f' % tuple(popt))
                x_val_pp = x_val_p[:]
                for j in range(1, 7):
                    x_val_pp.append(x_val_p[-1] + j)
                
                if len(x_val_p) == 2:
                    label_to_add = '(Unreliable prediction due to a small number of fitting points)\n'
                elif len(x_val_p) == 3:
                    label_to_add = '(Relatively reliable prediction)\n'
                elif len(x_val_p) > 3:
                    label_to_add = '(Reliable prediction)\n'
                else:
                    print('Number of points in x_val_p is unexpectedly below 2 (alarm ID ', al.iloc[m, 0], ')') #this condition only for error tracing
                y_pred = []
                dev_lower = 0
                dev_higher = 0
                dev_lower_list = []
                dev_higher_list = []
                range_val = 0
                range_list = []
                for o in range(len(x_val_p), len(x_val_pp)):
                    y_pred.append(popt[1] + (T0 - popt[1])*np.exp(-popt[0]*o))
                    if y_pred[-1] >= higher:
                        dev_higher = 10*(o - len(x_val_p) + 1)
                        dev_higher_list.append(dev_higher)
                    elif y_pred[-1] <= lower:
                        dev_lower = 10*(o - len(x_val_p) + 1)
                        dev_lower_list.append(dev_lower)
                    elif y_pred[-1] < higher and y_pred[-1] > lower:
                        range_val = 10*(o - len(x_val_p) + 1)
                        range_list.append(range_val)
                    else:
                        print('Something went wrong with the comparison of predicted temperature with the thresholds') #this condition only for error tracing
                if len(range_list) == 0:
                    range_list.append(0)
                if len(dev_lower_list) == 0:
                    dev_lower_list.append(0)
                if len(dev_higher_list) == 0:
                    dev_higher_list.append(0)
                if min(range_list) > max(dev_lower_list) and min(range_list) > max(dev_higher_list): #temperature will be within range
                    if (len(dev_lower_list) == 1 and dev_lower_list[0] == 0) and (len(dev_higher_list) == 1 and dev_higher_list[0] == 0):
                        label_to_add += 'Temperature is expected to remain within the range within the next 60 minutes'
                        if abs(y_pred[-1] - higher) <= 0.5:
                            label_to_add += ',\nhowever, it may be very close to higher threshold'
                        elif abs(y_pred[-1] - lower) <= 0.5:
                            label_to_add += ',\nhowever, it may be very close to lower threshold'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                    else:
                        label_to_add += 'Temperature is expected to be within range in ' + str(min(range_list)) + ' minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                elif min(dev_lower_list) > max(range_list) and min(dev_lower_list) > max(dev_higher_list):
                    if ((len(range_list) == 1 and range_list[0] == 0) and (len(dev_higher_list) == 1 and dev_higher_list[0] == 0)):
                        label_to_add += 'Temperature is expected to remain below lower threshold within the next 60 minutes'
                        if popt[-1] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                    else:
                        label_to_add += 'Temperature is expected to go below lower threshold in ' + str(min(dev_lower_list)) + ' minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                elif min(dev_higher_list) > max(dev_lower_list) and min(dev_higher_list) > max(range_list):
                    if ((len(dev_lower_list) == 1 and dev_lower_list[0] == 0) and (len(range_list) == 1 and range_list[0] == 0)):
                        label_to_add += 'Temperature is expected to remain beyond higher threshold within the next 60 minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                    else:
                        label_to_add += 'Temperature is expected to go beyond higher threshold in ' + str(min(dev_higher_list)) + ' minutes'
                        if popt[0] < 0.5:
                            label_to_add = label_to_add + ';\ntemperature is not likely to converge within one hour'
                        else:
                            label_to_add = label_to_add + ';\ntemperature is likely to converge within one hour and will equal ' + str(round(y_pred[-1], 2))
                
                plt.plot(x_val_pp[(len(x_val_p)):], temp_plot(np.array(x_val_pp), T0, *popt)[(len(x_val_p)):], 'r^', label = label_to_add)
                plt.legend(loc = 'best')
                plt.suptitle('Points processed in total: ' + str(len(input_y[:(i+1)])) + '; alarm ID: ' + str(al.iloc[m, 0]))
                plt.savefig('AlarmID %i Points %i.png' % (al.iloc[m, 0], len(input_y[:(i+1)])), dpi=100)
                plt.close()
                plt.rcParams.update({'figure.max_open_warning': 0})
            except RuntimeError:
                print('Optimal paramteres not found for ', y_temp)
            y_temp.append(input_y[i+1])
            x_val_p = np.arange(len(y_temp))
            x_val_p = list(x_val_p)
            plot_dots = 2
        else:
            print("Something went wrong")

/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:787: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
